<a href="https://colab.research.google.com/github/mhqadi/ML_alqadi/blob/main/docs/examples/train_object_detection_modeljo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train an Object Detection Model with GeoAI

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/train_object_detection_model.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [2]:
 %pip install geoai-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.3 MB/s eta 0:00:00
  

## Import libraries

In [1]:
import geoai

## Download sample data

In [8]:
train_raster_url= (
    "/content/Sentinel2_Amman_10mBands.tif"
)
train_vector_url= "/content/hafir.json"
test_raster_url= (
    "/content/Sentinel2_Specified_Area_10mBands.tif"
)

In [9]:
train_raster_path= geoai.download_file(train_raster_url)
train_vector_path= geoai.download_file(train_vector_url)
test_raster_path= geoai.download_file(test_raster_url)

File already exists: Sentinel2_Amman_10mBands.tif
File already exists: hafir.json
File already exists: Sentinel2_Specified_Area_10mBands.tif


## Visualize sample data

In [10]:
geoai.view_vector_interactive(train_vector_path, tiles=train_raster_url)

In [11]:
geoai.view_raster(test_raster_url)

## Create training data

In [12]:
out_folder = "output"
tiles = geoai.export_geotiff_tiles(
    in_raster=train_raster_path,
    out_folder=out_folder,
    in_class_data=train_vector_path,
    tile_size=32,
    stride=16,
    buffer_radius=0,
)


Raster info for Sentinel2_Amman_10mBands.tif:
  CRS: EPSG:4326
  Dimensions: 3341 x 2228
  Resolution: (8.983152841195215e-05, 8.983152841195215e-05)
  Bands: 4
  Bounds: BoundingBox(left=35.79993019731641, bottom=31.849948061514457, right=36.100057333740736, top=32.05009270681629)
Loaded 40 features from hafir.json
Vector CRS: EPSG:4326


Generated: 28912, With features: 0: 100%|██████████| 28912/28912 [05:17<00:00, 91.09it/s]


------- Export Summary -------
Total tiles exported: 28912
Tiles with features: 0 (0.0%)
Output saved to: output

------- Georeference Verification -------


## Train object detection model

In [1]:
geoai.train_MaskRCNN_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/models",
    num_channels=4,
    pretrained=True,
    batch_size=4,
    num_epochs=10,
    learning_rate=0.0005,
    val_split=0.2,
)

NameError: name 'geoai' is not defined

## Run inference

In [ ]:
masks_path = "Amman_test_prediction.tif"
model_path = f"{out_folder}/models/best_model.pth"

In [ ]:
geoai.object_detection(
    test_raster_path,
    masks_path,
    model_path,
    window_size=32,
    overlap=16,
    confidence_threshold=0.5,
    batch_size=4,
    num_channels=4,
)

## Vectorize masks

In [ ]:
output_path = "Amman_test_prediction.geojson"
gdf = geoai.orthogonalize(masks_path, output_path, epsilon=2)

## Visualize results

In [ ]:
geoai.view_vector_interactive(output_path, tiles=test_raster_url)

In [ ]:
geoai.create_split_map(
    left_layer=output_path,
    right_layer=test_raster_url,
    left_args={"style": {"color": "red", "fillOpacity": 0.2}},
    basemap=test_raster_url,
)

![image](https://github.com/user-attachments/assets/8dfcc69e-7a6c-408a-9fae-10b81b7d85dc)